##Text cleaning###

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
import pandas as pd
import numpy as np
from sutime import SUTime


import pickle as pk

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import neuralcoref
from pathlib import Path
from email import policy
from email.parser import BytesParser
import json
import nltk.data
import os
import spacy
from nltk.tokenize import WhitespaceTokenizer
import csv
from nltk import sent_tokenize
import numpy as np
import pandas as pd

In [ ]:
def remove_html_tags_func(text):
    '''
    Removes HTML-Tags from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without HTML-Tags
    ''' 
    return BeautifulSoup(text, 'html.parser').get_text()
def remove_url_func(text):
    '''
    Removes URL addresses from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without URL addresses
    ''' 
    return re.sub(r'https?://\S+|www\.\S+', '', text)
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

def relativedate(text,absdate):
    sutime = SUTime(mark_time_ranges=True, include_range=True)
    
    with open('timerelative', 'w') as fout:
        json.dump(sutime.parse(text,absdate), sort_keys=True, indent=4)
    
    

In [ ]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref is not None:
                if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                    if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                        tok_list[coref.start] = cluster.main.text + \
                            doc[coref.end-1].whitespace_
                        for i in range(coref.start+1, coref.end):
                            tok_list[i] = ""

    return "".join(tok_list)

In [ ]:
def textcleaning(text):
    text_html = remove_html_tags_func(text)
    text_url = remove_url_func(text_html) 
    text_acc = remove_accented_chars_func(text_url)
    #text_punct = remove_punctuation_func(text_acc)
    #text_irr = remove_irr_char_func(text_acc)   
    text_clean = remove_extra_whitespaces_func(text_acc)
   
    return text_clean

In [ ]:
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)


user_input = input('What is the name of your directory')
directory = os.listdir(user_input)

emailno=0
file = open("filejun23.csv", "w")
#f1 = open("inputjun23.txt", "w")
for fname in directory:
        if os.path.isfile(user_input + os.sep + fname):
       
            with open(user_input + os.sep + fname, 'rb') as f:
                name = f.name  # Get file name
                msg = BytesParser(policy=policy.default).parse(f)
                absdate = msg['Date']
            print(name)
            
                #preferencelist=('plain')
            if (msg.get_body(preferencelist=('plain'))):
                emailno=emailno+1
                print(emailno)
                text = msg.get_body(preferencelist=('plain')).get_content()
            
                text_content = coref_resolution(text)
                
                text_content = textcleaning(text_content)
                punctuations = '''“'!()—-―[]{};:'"\<>/?@#$%^&*_~'''
                
                no_punct = ""
                for char in text_content:
                    if char not in punctuations:
                        no_punct = no_punct + char
                        
                text_content = no_punct 
                #relativedate(text_content, absdate)
                #f1.write(text_content)
                #wrdcnt = word_count_func(text_content)
                #f= open("processed"+fname+".txt","w+")
                #f.write(text_content)
                #f.close()
                print(fname)
                applyNER(emailno, fname, text_content)
                #NER(text_content,emailno,name)
                #ner_new(text_content,emailno,name)

In [ ]:
def applyNER(emailno,fname,text):
#text = ("BILL IS TRAVELLING FROM CHICAGO TO DALLAS THIS SUNDAY.")
    
    
    text=remove_extra_whitespaces_func(text)
    sent= text.split('.')
    for t in sent:
        text1=coref_resolution(t)
    
    
    doc = nlp(text)
    words = []
    sente = []
    #labels = []
    sent_i =0
    
    for sent_i,sent in enumerate(doc.sents):
       
        for token in sent:
            if token.ent_type_:
            #print(token.text, token.ent_iob_, token.ent_type_)
                words.append(token.text)
                sente.append(sent_i)
            #iob.append(token.ent_iob_)
                #labels.append(token.ent_iob_+'-'+token.ent_type_)
                
                #labels.append()
            else:
                words.append(token.text)
                sente.append(sent_i)
            #iob.append()
                #labels.append(token.ent_iob_)
            #print(token.text, token.ent_iob_, 'NULL')
    df = pd.DataFrame({'emailno' : fname, 'sentno': sente, 'word': words, 'label': 'O'})
    #df.to_csv(r"{}.csv".format(fname), index=False) # biluo in extension to indicate the type of encoding, it is ok to keep csv

    df.to_csv('filejune23.csv', mode='a', index=False, header=False)
    #path = r"{}.csv".format(fname)

In [ ]:
user_input = input('What is the name of your directory')
directory = os.listdir(user_input)

emailno=0

file = open("filejun23.csv", "w")
#f1 = open("inputjun23.txt", "w")
for fname in directory:
        if os.path.isfile(user_input + os.sep + fname):
       
            with open(user_input + os.sep + fname, 'rb') as f:
                name = f.name  # Get file name
                msg = BytesParser(policy=policy.default).parse(f)
                absdate = msg['Date']
            print(name)
            
                #preferencelist=('plain')
            if (msg.get_body(preferencelist=('plain'))):
                emailno=emailno+1
                print(emailno)
                text = msg.get_body(preferencelist=('plain')).get_content()
            
                text_content = coref_resolution(text)
                print(text_content)
                text_content = textcleaning(text_content)
                punctuations = '''“'!()—-―[]{};:'"\<>/?@#$%^&*_~'''
                
                no_punct = ""
                for char in text_content:
                    if char not in punctuations:
                        no_punct = no_punct + char
                        
                text_content = no_punct 
                #relativedate(text_content, absdate)
                #f1.write(text_content)
                #wrdcnt = word_count_func(text_content)
                #f= open("processed"+fname+".txt","w+")
                #f.write(text_content)
                #f.close()
                print(fname)
                applyNER(emailno, fname, text_content)